怎么才能让你的电脑有那么一丢丢的小智能呢，现在好像有好多方法，但是那些高大上的方法，好像我们现在难以自己去实现，不过嘛万丈高楼平地起，我们可以用我们的聪明才智来实现一些小小的智能。

# 基础理论

那么我们就从手写数字识别开始吧！

我们都知道人工智能三要素是什么呢？数据、算法、计算资源。现在呢我们需要第一个东西“数据”。那数据在哪呢，原遇到，等等，楼门口来前辈们为我们贴心的准备好了，他就藏在一个叫keras的库里面。那么数据解决了算法呢？算法自在心中，下面就让我来细细说说吧。

这里呢我们就用一个结构简单，且易于理解的CNN卷积神经网络。什么！CNN！不就是那个图里面一层一层好复杂的东西吗？比如下面这个东西。莫怕莫怕，且听我细细道来
![图片](./image.png "aaa")

其实呢，抛去那些晦涩难懂的概念，细说CNN的构成就那么几层罢了，卷积层、激活函数、池化层、全连接层。我们这里打算仅搭建四层，进行这个让我们的电脑成为一个“小超人”这个神圣的任务。第一层：卷积层 、第二层：卷积层、 第三层：全连接层、 第四层：输出层。

我们的数据集呢「1」，文末介绍。我们建立这些层的原因是什么呢？这要从卷积神经网络的原理说起，即对图片的特征进行提取，我们现在第二个卷积层窗口大小为5×5，对32张图像求卷积产生64个特征图，参数个数是 5×5×32×64=51200个权值加上64个偏置。

池化计算是在卷积层中进行的，使用2×2，步长为2的池化窗口做池化计算，池化后得到64张7×7的特征图。特征图长宽都变成了之前的1/2。

第三层是全连接层，为池化层的结果做池化计算，池化后得到特征图。

第四层是输出层，输出预测值。

特征图数量越多说明卷积网络提取的特征数量越多，如果特征图数量设置得太少容易出现欠拟合，如果特征图数量设置得太多容易出现过拟合，所以需要设置为合适的数值。 

# 开始搭建神经网络

In [10]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Convolution2D,MaxPooling2D,Flatten
from tensorflow.keras.optimizers import Adam


## 引入数据集

这里呢我们引入数据集是一个非常简单的事情，只需要导入`keras.datasets`下的`mnist`即可。

In [11]:
mnist = tf.keras.datasets.mnist
(train_data, train_target), (test_data, test_target) = mnist.load_data()

## 数据预处理

这里我们要使用TensorFlow来进行后续操作，所以需要先对数据进行处理。

In [12]:
# 2、改变数据维度
train_data = train_data.reshape(-1, 28, 28, 1)
test_data = test_data.reshape(-1, 28, 28, 1)
# 注：在TensorFlow中，在做卷积的时候需要把数据变成4维的格式
# 这4个维度分别是：数据数量，图片高度，图片宽度，图片通道数

# 3、归一化（有助于提升训练速度）
train_data = train_data/255.0
test_data = test_data/255.0

# 4、独热编码
train_target = tf.keras.utils.to_categorical(train_target, num_classes=10)
test_target = tf.keras.utils.to_categorical(test_target, num_classes=10)    #10种结果

## 搭建网络

In [13]:
# 5、搭建CNN卷积神经网络
model = Sequential()

### 卷积层的搭建

这一层主要是由卷积层+池化层组成，在tensorflow中为我们直接提供了函数。

In [14]:
# 5-1、第一层：卷积层+池化层
# 第一个卷积层
model.add(Convolution2D(input_shape = (28,28,1), filters = 32, kernel_size = 5, strides = 1, padding = 'same', activation = 'relu'))
#         卷积层         输入数据                  滤波器数量      卷积核大小        步长          填充数据(same padding)  激活函数
# 第一个池化层 # pool_size
model.add(MaxPooling2D(pool_size = 2, strides = 2, padding = 'same',))
#         池化层(最大池化) 池化窗口大小   步长          填充方式

### 第二个卷积层

In [15]:
# 5-2、第二层：卷积层+池化层
# 第二个卷积层
model.add(Convolution2D(64, 5, strides=1, padding='same', activation='relu'))
# 64:滤波器个数      5:卷积窗口大小
# 第二个池化层
model.add(MaxPooling2D(2, 2, 'same'))

In [16]:
# 5-3、扁平化 （相当于把(64,7,7,64)数据->(64,7*7*64)）
model.add(Flatten())

### 第一个全连接层

In [17]:
# 5-4、第三层：第一个全连接层
model.add(Dense(1024, activation = 'relu'))
model.add(Dropout(0.5))

### 第二个全连接层（输出层）

In [18]:
# 5-5、第四层：第二个全连接层（输出层）
model.add(Dense(10, activation='softmax'))
# 10：输出神经元个数

### 编译

In [19]:
model.compile(optimizer=Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

### 训练与保存

In [20]:
model.fit(train_data, train_target, batch_size=64, epochs=10, validation_data=(test_data, test_target))
model.save("mnist.h5")

Epoch 1/10
938/938 [==============================] - 7s 7ms/step - loss: 0.3276 - accuracy: 0.9067 - val_loss: 0.0909 - val_accuracy: 0.9723
Epoch 2/10
938/938 [==============================] - 6s 6ms/step - loss: 0.0939 - accuracy: 0.9713 - val_loss: 0.0546 - val_accuracy: 0.9832
Epoch 3/10
938/938 [==============================] - 6s 7ms/step - loss: 0.0677 - accuracy: 0.9791 - val_loss: 0.0437 - val_accuracy: 0.9856
Epoch 4/10
938/938 [==============================] - 6s 7ms/step - loss: 0.0535 - accuracy: 0.9840 - val_loss: 0.0353 - val_accuracy: 0.9885
Epoch 5/10
938/938 [==============================] - 6s 7ms/step - loss: 0.0435 - accuracy: 0.9864 - val_loss: 0.0300 - val_accuracy: 0.9895
Epoch 6/10
938/938 [==============================] - 6s 6ms/step - loss: 0.0367 - accuracy: 0.9884 - val_loss: 0.0265 - val_accuracy: 0.9908
Epoch 7/10
938/938 [==============================] - 6s 7ms/step - loss: 0.0317 - accuracy: 0.9902 - val_loss: 0.0289 - val_accuracy: 0.9896
Epoch 